<a href="https://colab.research.google.com/github/vedantg-1311/Ru-Bioactivity-Classification/blob/main/Ru_NNFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Application of Neural Networks to Classify Transition Metal Complexes of Ruthenium

## Installing libraries and dataset

In [ ]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 26.1 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/TheFreiLab/RutheniumML.git

Cloning into 'RutheniumML'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 31 (delta 4), reused 26 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (31/31), 1.49 MiB | 6.67 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [ ]:
import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs, Draw

from sklearn.model_selection import train_test_split

## Data Pre-processing

In [ ]:
arenes = pd.read_csv('RutheniumML/data/tested_arenes.csv', dtype={'ID': str, 'SMILES': str})
amines = pd.read_csv('RutheniumML/data/tested_amines.csv', dtype={'ID': str, 'SMILES': str})
aldehydes = pd.read_csv('RutheniumML/data/tested_aldehydes.csv', dtype={'ID': str, 'SMILES': str})
actives = pd.read_csv('RutheniumML/data/actives.csv', dtype={'ID': str, 'MIC': float, 'CC50': float, 'HC10': float})
actives['IsActive'] = 1

arenes['MOL'] = arenes.SMILES.apply(Chem.MolFromSmiles)
amines['MOL'] = amines.SMILES.apply(Chem.MolFromSmiles)
aldehydes['MOL'] = aldehydes.SMILES.apply(Chem.MolFromSmiles)

def calc_ecfp4(mol):
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=512)
    arr = np.zeros((0,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp,arr)
    return arr

arenes['ECFP4'] = arenes.MOL.apply(calc_ecfp4)
amines['ECFP4'] = amines.MOL.apply(calc_ecfp4)
aldehydes['ECFP4'] = aldehydes.MOL.apply(calc_ecfp4)

id = []
smiles = []
ecfp4 = []

for i in range(len(arenes)):
    for j in range(len(amines)):
        for k in range(len(aldehydes)):
            id.append(f'{arenes.ID.iloc[i]}{amines.ID.iloc[j]}{aldehydes.ID.iloc[k]}')
            smiles.append(f'{arenes.SMILES.iloc[i]}.{amines.SMILES.iloc[j]}.{aldehydes.SMILES.iloc[k]}')
            ecfp4.append(np.sum([arenes.ECFP4.iloc[i], amines.ECFP4.iloc[j], aldehydes.ECFP4.iloc[k]], axis=0)) # ECFPs are summed

df_dataset = pd.DataFrame(list(zip(id, smiles, ecfp4)), columns=['ID', 'SMILES', 'ECFP4'])

df_train = df_dataset.copy()
actives_id = actives.ID.values.tolist()

def IsActive(id, id_list):
    if id in id_list:
        return 1
    else:
        return 0

df_train['IsActive'] = df_train.ID.apply(lambda x: IsActive(x, actives_id))

X = np.array(df_train.ECFP4.values.tolist())
y = np.array(df_train.IsActive.values.tolist())

df_actives_only = df_train.loc[df_train['IsActive'] == True]
df_inactives_only = df_train.loc[df_train['IsActive'] == False]
X_actives_only = np.array(df_actives_only.ECFP4.values.tolist())
X_inactives_only = np.array(df_inactives_only.ECFP4.values.tolist())
y_actives_only = np.array(df_actives_only.IsActive.values.tolist())
y_inactives_only = np.array(df_inactives_only.IsActive.values.tolist())

X_actives_train, X_actives_test, y_actives_train, y_actives_test = train_test_split(X_actives_only, y_actives_only, test_size=0.20, random_state=1)
X_inactives_train, X_inactives_test, y_inactives_train, y_inactives_test = train_test_split(X_inactives_only, y_inactives_only, test_size=0.20, random_state=1)

y_train_final = np.append(y_actives_train, y_inactives_train)
X_train_final = np.append(X_actives_train, X_inactives_train, axis=0)

[08:15:31] DEPRECATION WARNING: please use MorganGenerator
[08:15:31] DEPRECATION WARNING: please use MorganGenerator
[08:15:31] DEPRECATION WARNING: please use MorganGenerator
[08:15:31] DEPRECATION WARNING: please use MorganGenerator
[08:15:31] DEPRECATION WARNING: please use MorganGenerator
[08:15:31] DEPRECATION WARNING: please use MorganGenerator
[08:15:31] DEPRECATION WARNING: please use MorganGenerator
[08:15:31] DEPRECATION WARNING: please use MorganGenerator
[08:15:31] DEPRECATION WARNING: please use MorganGenerator
[08:15:31] DEPRECATION WARNING: please use MorganGenerator
[08:15:31] DEPRECATION WARNING: please use MorganGenerator
[08:15:31] DEPRECATION WARNING: please use MorganGenerator
[08:15:31] DEPRECATION WARNING: please use MorganGenerator
[08:15:31] DEPRECATION WARNING: please use MorganGenerator
[08:15:31] DEPRECATION WARNING: please use MorganGenerator
[08:15:31] DEPRECATION WARNING: please use MorganGenerator
[08:15:31] DEPRECATION WARNING: please use MorganGenerat

## Neural Network Architecture

In [ ]:
class Layer_Dense:
  def __init__(self, n_inputs, n_neurons):
    self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
    self.biases = np.zeros((1, n_neurons))

  def forward(self, inputs):
    self.inputs = inputs
    self.output = np.dot(inputs, self.weights) + self.biases

  def backward(self, dvalues):
    self.dweights = np.dot(self.inputs.T, dvalues)
    self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
    self.dinputs = np.dot(dvalues, self.weights.T)

class Activation_ReLU:
  def forward(self, inputs):
    self.inputs = inputs
    self.output = np.maximum(0, inputs)

  def backward(self, dvalues):
    self.dinputs = dvalues.copy()
    self.dinputs[self.inputs <= 0] = 0

class Activation_Softmax:
  def forward(self, inputs):
    exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True)) #????????????????????????????????????????????
    probabilities = exp_values/np.sum(exp_values, axis=1, keepdims=True)
    self.output = probabilities

class Loss:
  def calculate(self, output, y):
    sample_losses = self.forward(output, y)
    data_loss = np.mean(sample_losses)
    return data_loss

class Loss_CategoricalCrossentropy(Loss):
  def forward(self, y_pred, y_true):
    samples = len(y_pred)
    y_pred_clipped  = np.clip(y_pred, 1e-7, 1-1e-7)

    if len(y_true.shape) == 1:
      correct_confidences = y_pred_clipped[range(samples), y_true] #???????????????????????????????

    elif len(y_true.shape) == 2:
      correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

    negative_log_likelihoods = -np.log(correct_confidences)
    return negative_log_likelihoods

  def backward(self, dvalues, y_true):
    samples = len(dvalues)
    labels = len(dvalues[0])

    if len(y_true.shape) == 1:
      y_true = np.eye(labels)[y_true] #?????????????????

    self.dinputs = -y_true / dvalues
    self.dinputs = self.dinputs / samples

class Activation_Softmax_Loss_CategoricalCrossentropy:
  def __init__(self):
    self.activation = Activation_Softmax()
    self.loss = Loss_CategoricalCrossentropy()

  def forward(self, inputs, y_true):
    self.activation.forward(inputs)
    self.output = self.activation.output
    return self.loss.calculate(self.output, y_true)

  def backward(self, dvalues, y_true):
    samples = len(dvalues)
    if len(y_true.shape) == 2:
      y_true = np.argmax(y_true, axis=1)
    self.dinputs = dvalues.copy()
    self.dinputs[range(samples), y_true] -= 1
    self.dinputs = self.dinputs / samples

class Optimizer_Adam:
  def __init__(self, learning_rate=0.001, decay=0., epsilon=1e-7, beta_1=0.9, beta_2=0.999):
    self.learning_rate = learning_rate
    self.current_learning_rate = learning_rate
    self.decay = decay
    self.iterations = 0
    self.epsilon = epsilon
    self.beta_1 = beta_1
    self.beta_2 = beta_2

  def pre_update_params(self):
    if self.decay:
      self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

  def update_params(self, layer):
    if not hasattr(layer, 'weight_cache'):
      layer.weight_momentums = np.zeros_like(layer.weights)
      layer.weight_cache = np.zeros_like(layer.weights)
      layer.bias_momentums = np.zeros_like(layer.biases)
      layer.bias_cache = np.zeros_like(layer.biases)

    layer.weight_momentums = self.beta_1 * layer.weight_momentums + (1 - self.beta_1) * layer.dweights
    layer.bias_momentums = self.beta_1 * layer.bias_momentums + (1 - self.beta_1) * layer.dbiases

    weight_momentums_corrected = layer.weight_momentums / (1 - self.beta_1 ** (self.iterations + 1))
    bias_momentums_corrected = layer.bias_momentums / (1 - self.beta_1 ** (self.iterations + 1))

    layer.weight_cache = self.beta_2 * layer.weight_cache + (1 - self.beta_2) * layer.dweights**2
    layer.bias_cache = self.beta_2 * layer.bias_cache + (1 - self.beta_2) * layer.dbiases**2

    weight_cache_corrected = layer.weight_cache / (1 - self.beta_2 ** (self.iterations + 1))
    bias_cache_corrected = layer.bias_cache / (1 - self.beta_2 ** (self.iterations + 1))

    layer.weights += -self.current_learning_rate * weight_momentums_corrected / (np.sqrt(weight_cache_corrected) + self.epsilon)
    layer.biases += -self.current_learning_rate * bias_momentums_corrected / (np.sqrt(bias_cache_corrected) + self.epsilon)

  def post_update_params(self):
    self.iterations += 1

## Training

In [ ]:
dense1 = Layer_Dense(512, 400)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(400, 300)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

optimizer = Optimizer_Adam(learning_rate=0.02, decay=1e-5)

for epoch in range(1001):
  dense1.forward(X_train_final)
  activation1.forward(dense1.output)
  dense2.forward(activation1.output)
  loss = loss_activation.forward(dense2.output, y_train_final)

  predictions = np.argmax(loss_activation.output, axis=1)
  if len(y.shape) == 2:
    y = np.argmax(y, axis=1)
  accuracy = np.mean(predictions == y_train_final)

  if not epoch % 100:
    print(f'epoch: {epoch}, ' +
          f'acc: {accuracy:.3f}, ' +
          f'loss: {loss:.3f}, ' +
          f'lr: {optimizer.current_learning_rate}')

  loss_activation.backward(loss_activation.output, y_train_final)
  dense2.backward(loss_activation.dinputs)
  activation1.backward(dense2.dinputs)
  dense1.backward(activation1.dinputs)

  optimizer.pre_update_params()
  optimizer.update_params(dense1)
  optimizer.update_params(dense2)
  optimizer.post_update_params()

epoch: 0, acc: 0.000, loss: 5.703, lr: 0.02
epoch: 100, acc: 1.000, loss: 0.006, lr: 0.01998021958261321
epoch: 200, acc: 1.000, loss: 0.001, lr: 0.019960279044701046
epoch: 300, acc: 1.000, loss: 0.000, lr: 0.019940378268975763
epoch: 400, acc: 1.000, loss: 0.000, lr: 0.01992051713662487
epoch: 500, acc: 1.000, loss: 0.000, lr: 0.01990069552930875
epoch: 600, acc: 1.000, loss: 0.000, lr: 0.019880913329158343
epoch: 700, acc: 1.000, loss: 0.000, lr: 0.019861170418772778
epoch: 800, acc: 1.000, loss: 0.000, lr: 0.019841466681217078
epoch: 900, acc: 1.000, loss: 0.000, lr: 0.01982180200001982
epoch: 1000, acc: 1.000, loss: 0.000, lr: 0.019802176259170884


## Validation

In [ ]:
X_test_final = np.append(X_actives_test, X_inactives_test, axis=0)
y_test_final = np.append(y_actives_test, y_inactives_test)

dense1.forward(X_test_final)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
loss = loss_activation.forward(dense2.output, y_test_final)
predictions = np.argmax(loss_activation.output, axis=1)
if len(y_test_final.shape) == 2:
 y_test = np.argmax(y_test_final, axis=1)
accuracy = np.mean(predictions == y_test_final)
print(f'validation, acc: {accuracy:.3f}, loss: {loss:.3f}')

validation, acc: 0.949, loss: 0.413
